In [1]:
#IMPORTS
import pm4py
import sklearn
import pandas as pd
import numpy as np
import torch

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from pm4py.algo.transformation.log_to_features import algorithm as log_to_features
from itertools import product

# Prepare the Data
The code below is used to generate the CVS file. You can also skip running it and instead load the CSV directly a few cells below.

First, make sure the permit log is in the current directory, then run the code below

In [2]:
travel_permits = pm4py.read_xes('PermitLog.xes.gz')

parsing log, completed traces ::   0%|          | 0/7065 [00:00<?, ?it/s]

First, we make sure we have the prefixes we need, we just want the events leading up to 'Start trip'

In [3]:
travel_prefixes = pm4py.filtering.filter_prefixes(travel_permits, 'Start trip')

InvalidVersion: Invalid version: 'please use pm4py.algo.filtering.log.ltl.ltl_checker.eventually_follows'

In [ ]:
#quick check to see if we got what we wanted
i = 39
for e in travel_permits[i]:
    print(e['concept:name'])
print() 
for e in travel_prefixes[i]:
    print(e['concept:name'])

Check if a declaration was submitted before the trip started

In [ ]:
decl_bools = np.zeros((7065,1))

for i,t in enumerate(travel_prefixes):
    for e in t:
        if e['concept:name'] == "Declaration SUBMITTED by EMPLOYEE":
            decl_bools[i,0] = 1
            
decl_bools.shape

Now extract the features we are interested in from the prefixes:

In [ ]:
data, _ = log_to_features.apply(travel_prefixes, parameters={'num_tr_attr': ["RequestedBudget","OverspentAmount"]})

data = np.array(data).round(2)
data.shape

However, we still need trip duration, which we will need to compute manually. We will use number of days to denote the duration

In [ ]:
trip_durations = []
for trace in travel_permits:
    for e in trace:
        if e['concept:name'] == "Start trip": #note the time the trip started
            start_time = e['time:timestamp']
        elif e['concept:name'] == "End trip": #note the time the trip ended
            end_time = e['time:timestamp']
            break
    trip_durations.append((end_time-start_time).days) #save the trip duration in days
    
trip_durations = np.array(trip_durations).reshape((-1,1))
trip_durations.shape

And we do the same in order to get the duration between the permit being submitted and the permit being approved

In [ ]:
perm_durations = []
for trace in travel_permits:
    started = False
    for e in trace:
        if e['concept:name'].startswith("Permit") and not started: #start time is the first time permit is mentioned
            started = True
            start_time = e['time:timestamp']
            end_time = e['time:timestamp']
        elif e['concept:name'].startswith("Permit"): #end time is the last time permit is mentioned
            end_time = e['time:timestamp']
    perm_durations.append((end_time-start_time).days) #save duration of permit handling in days
    
perm_durations = np.array(perm_durations).reshape((-1,1))
perm_durations.shape

Now we merge all the features we extracted into one dataframe

In [ ]:
final_data = np.concatenate((decl_bools, trip_durations, perm_durations, data), axis=1)
#switch the trip duration index with the overspent amount, since overspent amount is what we want to predict, so it's more intuitive
#final_data[:,[-1,-3]] = final_data[:,[-3,-1]]
final_data.shape

Some exploration of the data we have now:

In [ ]:
pd.DataFrame(data=final_data[:10,:])

We can now save the data to CSV:

In [ ]:
np.savetxt('ML_data2.csv', final_data, delimiter=',')

# Load the CSV
Or skip the below cell if you already ran the above cells

In [4]:
final_data = np.genfromtxt('ML_data2.csv',delimiter=',')
final_data.shape

(7065, 5)

# Train the ML model
First, we split into train, test and validation sets. We do not shuffle because for process mining it is important that the validation and test sets are further in the future than the training set. The training set is 75%, validation 12.5%, and test set 12.5%.

In [5]:
X_train, X_valtest, y_train, y_valtest = sklearn.model_selection.train_test_split(final_data[:,:-1], final_data[:,-1], 
                                                                                  train_size=0.75, shuffle=False)
X_val, X_test, y_val, y_test = sklearn.model_selection.train_test_split(X_valtest, y_valtest, train_size=0.5, shuffle=False)

Next, we do a parameter search and choose the best parameters based on performance on the validation set. The score metric here has a maximum value of 1.0 and no minimum value.

In [6]:
#no gridsearchcv because it doesn't allow for a validation set
best_score = -10000
for n_est, min_split in product([25,50,100,200], [1,20,50]):
    model = RandomForestRegressor(n_estimators = n_est, min_samples_split = min_split)
    model.fit(X_train, y_train)
    score = model.score(X_val, y_val)
    print("n_estimators: {}, min_samples_split: {}, score: {}".format(n_est, min_split, score))
    if score > best_score:
        best_score = score
        best_param = n_est, min_split

n_estimators: 25, min_samples_split: 1, score: -441.75531567269707
n_estimators: 25, min_samples_split: 20, score: -172.83937114910827
n_estimators: 25, min_samples_split: 50, score: -27.34968290240855
n_estimators: 50, min_samples_split: 1, score: -209.01813180308832
n_estimators: 50, min_samples_split: 20, score: -208.9923382991645
n_estimators: 50, min_samples_split: 50, score: -7.816657287969825
n_estimators: 100, min_samples_split: 1, score: -155.8107583862892
n_estimators: 100, min_samples_split: 20, score: -291.5959409721682
n_estimators: 100, min_samples_split: 50, score: -8.329258952731115
n_estimators: 200, min_samples_split: 1, score: -228.116444481496
n_estimators: 200, min_samples_split: 20, score: -208.91534192085655
n_estimators: 200, min_samples_split: 50, score: -7.095602384430771


Now we take the model with the best performing parameters and test its performance on the test set

In [7]:
best_model = RandomForestRegressor(n_estimators = best_param[0], min_samples_split = best_param[1])
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)

-7.047660646121992

This score metric isn't the most intuitive, so let's compute mean absolute error on the test set as well.

In [8]:
mean_absolute_error(y_test, best_model.predict(X_test))

924.2105526147691

In [10]:
## Model Definition ##
import torch
from torchsummary import summary

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device=torch.device('cpu')

In [19]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        input_layer = torch.nn.Linear(4,16)
        hidden_layer = torch.nn.Linear(16, 16)
        output_layer = torch.nn.Linear(16, 1)

    def forward(self, x):
        x = input_layer(x)
        x = hidden_layer(x)
        x = output_layer(x)
        return x

model = Model().to(device)
#summary(model, input_size=(1,4))

In [20]:
## LOSS Definition ##
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#, momentum=0.5, weight_decay=1e-5)

ValueError: optimizer got an empty parameter list

In [14]:
## Training ##
#Trainer class from the practical, adapted slightly because we do not have a validation set

class Trainer():
    def __init__(self,
                 model: torch.nn.Module,
                 device: torch.device,
                 criterion: torch.nn.Module,
                 optimizer: torch.optim.Optimizer,
                 epochs: int,
                 mean_train_losses,
                 mean_val_losses
                 ):
        
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.epochs = epochs
        self.mean_train_losses = mean_train_losses
        self.mean_val_losses = mean_val_losses

    def run_trainer(self):
        for epoch in tqdm(range(self.epochs)):
            
            self.model.train()  # train mode
            #idx = [range(X_train.shape[0])]
            #batches = idx.split(self.batch_size)

            train_losses=[]
            for batch in range(X_train.shape[0]):
                x,y=X_train[batch], y_train[batch]
                input, target = x.to(self.device), y.to(self.device).reshape((-1,1))  # send to device (GPU or CPU)
                self.optimizer.zero_grad()  # zerograd the parameters
                out = self.model(input)  # one forward pass
                loss = self.criterion(out, target).cpu()  # calculate loss

                train_losses.append(float(loss))
                 
                loss.backward()  # one backward pass
                self.optimizer.step()  # update the parameters

            print(f'EPOCH: {epoch+1:0>{len(str(self.epochs))}}/{self.epochs}', end=' ')
            print(f"LOSS: {np.mean(train_losses):.4f}")
            self.mean_train_losses.append(np.mean(train_losses))
            
            val_losses=[]
            for batch in range(X_val.shape[0]):
                self.model.eval()
                x,y=X_val[batch], y_val[batch]
                input, target = x.to(self.device), y.to(self.device).reshape((-1,1))  # send to device (GPU or CPU)
                self.optimizer.zero_grad()  # zerograd the parameters
                out = self.model(input)  # one forward pass
                loss = self.criterion(out, target).cpu()  # calculate loss

                val_losses.append(float(loss))
                
            print(f'EPOCH: {epoch+1:0>{len(str(self.epochs))}}/{self.epochs}', end=' ')
            print(f"LOSS: {np.mean(val_losses):.4f}")
            self.mean_val_losses.append(np.mean(val_losses))


        def test(self):
            model = trainer.model.to("cpu")
            model.eval()
            test_losses=[]
            for batch in range(X_test.shape[0]):
                #self.model.eval()
                x,y=X_test[batch], y_test[batch]
                input, target = x.to(self.device), y.to(self.device).reshape((-1,1))  # send to device (GPU or CPU)
                self.optimizer.zero_grad()  # zerograd the parameters
                out = self.model(input)  # one forward pass
                loss = self.criterion(out, target).cpu()  # calculate loss

                test_losses.append(float(loss))
                
            print(f'EPOCH: {epoch+1:0>{len(str(self.epochs))}}/{self.epochs}', end=' ')
            print(f"LOSS: {np.mean(test_losses):.4f}")

In [15]:
trainer = Trainer(model=model,
                  device=device,
                  criterion=criterion,
                  optimizer=optimizer,
                  epochs=200,
                  mean_losses=[])

trainer.run_trainer()

NameError: name 'optimizer' is not defined

In [16]:
trainer.test()

NameError: name 'trainer' is not defined